# Налаштування моделі noobai-XL-1.1 на Google Colab

Цей блокнот налаштовує модель `noobai-XL-1.1` для генерації зображень на основі текстових описів з можливістю налаштування методів семплінгу, кількості кроків, насіння, масштабу CFG, розміру зображення та високоякісного масштабування.

In [1]:
# Встановлення необхідних бібліотек
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.31.0 diffusers==0.14.0 accelerate
!pip install git+https://github.com/huggingface/diffusers.git
!pip install --upgrade transformers
!pip install pillow

## Імпорт бібліотек

In [2]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
from diffusers import DDIMScheduler
import transformers
from PIL import Image
import os

## Конфігурація моделі та токенізатора

In [3]:
# Клонування репозиторію моделі
!git lfs install
!git clone https://huggingface.co/Laxhar/noobai-XL-1.1

# Модифікація конфігурації токенізатора для зняття обмеження символів
!sed -i 's/"max_length": 77/"max_length": 1024/' noobai-XL-1.1/tokenizer/tokenizer_config.json
!sed -i 's/"max_length": 77/"max_length": 1024/' noobai-XL-1.1/tokenizer_2/tokenizer_config.json

## Завантаження моделі

In [4]:
# Вибір пристрою (GPU або CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Налаштування семплера
scheduler = EulerAncestralDiscreteScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler")

# Завантаження пайплайну
pipe = StableDiffusionPipeline.from_pretrained(
    "Laxhar/noobai-XL-1.1",
    scheduler=scheduler,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True  # Переконайтесь, що у вас є доступ до моделі
).to(device)

## Визначення утилітних функцій

In [5]:
def generate_image(prompt, sampler='euler', steps=50, seed=None, cfg_scale=7.5, image_size=(512, 512)):
    
    # Встановлення семплера
    if sampler == 'euler':
        scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sampler == 'ddim':
        scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    # Додайте більше семплерів за потреби
    else:
        raise ValueError(f"Несподіваний семплер: {sampler}")
    
    pipe.scheduler = scheduler
    
    # Встановлення насіння для відтворюваності
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None
    
    # Генерація зображення
    with autocast(device):
        image = pipe(
            prompt,
            num_inference_steps=steps,
            guidance_scale=cfg_scale,
            height=image_size[1],
            width=image_size[0],
            generator=generator
        ).images[0]
    return image

## Виправлення високої роздільної здатності з 8x-NKMD-Superscale

In [6]:
# Клонування репозиторію SuperScale
!git clone https://github.com/NKMD-Development/superscale-8x.git

# Завантаження моделі SuperScale (змінити відповідно до фактичного використання моделі)
# Це заповнювач для фактичної реалізації SuperScale
# Можливо, вам потрібно встановити специфічні залежності або використовувати іншу бібліотеку
# Для демонстрації ми припустимо просте збільшення розміру зображення за допомогою PIL
def superscale_image(image, scale=8, creativity=1.0, steps=10):
    
    # Заповнювач для фактичного процесу SuperScale
    # Тут ми просто змінюємо розмір зображення як приклад
    new_size = (image.width * scale, image.height * scale)
    upscaled_image = image.resize(new_size, Image.LANCZOS)
    return upscaled_image

## Вхідні дані користувача

In [7]:
# Налаштування параметрів користувача
prompt = "Спокійний пейзаж з горами та річкою"
sampler = "euler"  # Опції: 'euler', 'ddim'
steps = 50
seed = 42
cfg_scale = 7.5
image_size = (512, 512)

# Параметри виправлення високої роздільної здатності
apply_superscale = True
scale_factor = 8
creativity = 1.0
superscale_steps = 10

## Генерація та відображення зображення

In [8]:
# Генерація зображення
image = generate_image(
    prompt=prompt,
    sampler=sampler,
    steps=steps,
    seed=seed,
    cfg_scale=cfg_scale,
    image_size=image_size
)

# Застосування виправлення високої роздільної здатності, якщо увімкнено
if apply_superscale:
    image = superscale_image(
        image,
        scale=scale_factor,
        creativity=creativity,
        steps=superscale_steps
    )

# Відображення зображення
image

<PIL.Image.Image image mode=RGB size=4096x4096 at 0x7F8C3C2E9C10>

## Збереження зображення

In [9]:
# Збереження зображення у локальну файлову систему
output_path = "generated_image.png"
image.save(output_path)
print(f"Зображення збережено за шляхом {output_path}")

## Додаткові ресурси

- [Hugging Face Diffusers](https://huggingface.co/docs/diffusers/index)
- [Документація Stable Diffusion](https://huggingface.co/docs/diffusers/stable_diffusion)
- [Документація Transformers](https://huggingface.co/docs/transformers/index)